In [1]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import rasterio

In [2]:
working_dir = (
    "Z:/landscape_geoinfo/2024_ditches/working/deep_learning/data/"
    "prediction"
)

In [3]:
# Read land use classes
land_use_classes = pd.read_csv(
    (
        "Z:/landscape_geoinfo/soilmap/working/1_peat_soils/ditches/"
        "landuse_classes.csv"
    )
)

In [4]:
land_use_classes

,landuse,landuse_int
0,arable,1
1,forest,2
2,other,3
3,peat_mining,4
4,wetland_bog,5
5,wetland_fen,6
6,wetland_marsh,7
7,wetland_quagmire,8


In [5]:
# Create dictionary of land use classes
land_use_classes_dict = dict(
    zip(land_use_classes["landuse_int"], land_use_classes["landuse"])
)

In [6]:
land_use_classes_dict

{1: 'arable',
 2: 'forest',
 3: 'other',
 4: 'peat_mining',
 5: 'wetland_bog',
 6: 'wetland_fen',
 7: 'wetland_marsh',
 8: 'wetland_quagmire'}

In [7]:
# Compute F1 score for specific land use class
def compute_f1_per_tile(
    true_labels, predicted_labels, land_use, land_use_class
):
    
    # Extract only relevant pixels
    mask = (land_use == land_use_class)
    y_true = true_labels[mask].flatten()
    y_pred = predicted_labels[mask].flatten()
    
    # Compute accuracy metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average="binary", zero_division=0
    )
    
    return precision, recall, f1, len(y_true)

In [8]:
# Probability threshold for predicted labels
prob_threshold = 0.5

In [9]:
%%time

# Loop over map sheets
map_sheets = [53972, 54591]
for map_sheet in map_sheets:
    
    # Read predicted labels
    fp_predicted_labels = (
        f"{working_dir}/"
        f"predicted_labels_{prob_threshold}/"
        f"{map_sheet}_predicted_labels_{prob_threshold}.tif"
    )
    with rasterio.open(fp_predicted_labels) as src:
        predicted_labels = src.read(1)
        
    # Read true labels
    fp_true_labels = (
        f"{working_dir}/"
        f"true_labels/{map_sheet}_true_labels.tif"
    )
    with rasterio.open(fp_true_labels) as src:
        true_labels = src.read(1)
        
    # Read land use
    fp_land_use = (
        f"{working_dir}/"
        f"{map_sheet}_landuse_reclassified.tif"
    )
    with rasterio.open(fp_land_use) as src:
        land_use = src.read(1)
        
    # Loop over land use classes
    results = []
    for class_value, class_name in land_use_classes_dict.items():
        
        # Compute accuracy metrics
        precision, recall, f1, pixel_count = compute_f1_per_tile(
            true_labels, predicted_labels, land_use, class_value
        )
        results.append(
            {
                "map_sheet": map_sheet,
                "prob_threshold": prob_threshold,
                "land_use": class_name,
                "precision": precision,
                "recall": recall,
                "f1": f1,
                "pixel_count": pixel_count
            }
        )
        
    # Create output DataFrame and write to CSV
    out_df = pd.DataFrame(results)
    display(out_df)
    out_df.to_csv(
        (
            f"{working_dir}/"
            f"{map_sheet}_f1_per_land_use_class_{prob_threshold}.csv"
        ),
        index=False
    )

,map_sheet,prob_threshold,land_use,precision,recall,f1,pixel_count
0,53972,0.5,arable,0.874597,0.610346,0.718959,874692
1,53972,0.5,forest,0.852100,0.644664,0.734008,7688983
2,53972,0.5,other,0.839895,0.672976,0.747227,87863
3,53972,0.5,peat_mining,0.000000,0.000000,0.000000,0
4,53972,0.5,wetland_bog,0.801653,0.218468,0.343363,283263
5,53972,0.5,wetland_fen,0.000000,0.000000,0.000000,0
6,53972,0.5,wetland_marsh,0.000000,0.000000,0.000000,4810
7,53972,0.5,wetland_quagmire,0.000000,0.000000,0.000000,0


,map_sheet,prob_threshold,land_use,precision,recall,f1,pixel_count
0,54591,0.5,arable,0.840140,0.726892,0.779424,3122777
1,54591,0.5,forest,0.848323,0.674895,0.751736,3727656
2,54591,0.5,other,0.825731,0.837485,0.831567,113555
3,54591,0.5,peat_mining,0.826526,0.817553,0.822015,1242887
4,54591,0.5,wetland_bog,0.725777,0.703901,0.714671,734399
5,54591,0.5,wetland_fen,0.000000,0.000000,0.000000,0
6,54591,0.5,wetland_marsh,0.000000,0.000000,0.000000,0
7,54591,0.5,wetland_quagmire,0.000000,0.000000,0.000000,0


CPU times: total: 5.62 s
Wall time: 15.9 s
